In [4]:
try :
  from google.colab import drive
  from google.colab import files
  drive.mount('/content/drive')
  PATH_GC = '/content/drive/My Drive' 
except Exception:
  print("Vous n'êtes pas sur Google Colab, vous êtes en local, assurez-vous d'avoir ajouter le Path du dossier Colab_sources")
  PATH_GC = './'#Emplacement du dossier /Colab_sources

Mounted at /content/drive


In [5]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn

In [8]:
X_train = pd.read_csv(PATH_GC+'/Challenge/X_train.csv', index_col=0, sep=',')
X_train.columns.name = 'date'

Y_train = pd.read_csv(PATH_GC+'/Challenge/Y_train.csv', index_col=0, sep=',')
Y_train.columns.name = 'date'

In [9]:
X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_train_reshape.columns = pd.Index(range(1,251), name='timeLag')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """Entry point for launching an IPython kernel.


In [10]:
X_train_reshape


timeLag             1         2         3         4         5         6    \
date stocksID                                                               
250  0         0.000103  0.012387  0.011243  0.002595 -0.008509 -0.002711   
     1        -0.000982  0.003932  0.000050  0.001616 -0.003902 -0.001686   
     2         0.009301  0.003914  0.004995  0.001539  0.001452  0.002809   
     3         0.006515 -0.006553  0.009464  0.005204  0.004227 -0.005438   
     4        -0.006223  0.005415  0.014643  0.005195  0.004489  0.002695   
...                 ...       ...       ...       ...       ...       ...   
753  45        0.009786 -0.001861  0.012294 -0.023981  0.004069  0.000148   
     46       -0.008842  0.003698  0.005138 -0.004295 -0.010509  0.010230   
     47       -0.008607  0.006122 -0.007828 -0.008224  0.011357 -0.012855   
     48       -0.007519  0.001956  0.001903  0.001535  0.000695  0.004107   
     49       -0.002816 -0.002696 -0.007145 -0.001814  0.004331  0.005457   

timeLag             7         8         9         10   ...       241  \
date stocksID                                          ...             
250  0         0.008934  0.006571 -0.018546 -0.008353  ...  0.009119   
     1         0.008810  0.001585 -0.000745 -0.002155  ...  0.001664   
     2         0.005177 -0.006942 -0.013340 -0.008071  ...  0.007416   
     3         0.008861  0.004025 -0.012432 -0.006100  ...  0.003807   
     4         0.007609  0.011437 -0.004804  0.039274  ...  0.014404   
...                 ...       ...       ...       ...  ...       ...   
753  45       -0.062300  0.040002 -0.003475 -0.016284  ...  0.003808   
     46        0.006846  0.017514 -0.010796 -0.026356  ... -0.012077   
     47        0.000346 -0.016681  0.008247 -0.002723  ...  0.007291   
     48       -0.011879  0.000526  0.003178 -0.010654  ...  0.010960   
     49        0.000633 -0.005819  0.007592 -0.002003  ... -0.024003   

timeLag             242       243       244       245       246       247  \
date stocksID                                                               
250  0        -0.008451  0.007120 -0.011745  0.009092 -0.005110 -0.016676   
     1        -0.002747  0.009994 -0.001293  0.012912 -0.004055 -0.006820   
     2         0.001600  0.007640 -0.000637 -0.001353 -0.009522 -0.003425   
     3        -0.010636  0.012446 -0.011980  0.002485 -0.017981 -0.010899   
     4        -0.005255 -0.008182 -0.004614  0.007082  0.007872 -0.003515   
...                 ...       ...       ...       ...       ...       ...   
753  45       -0.003027  0.002522 -0.006594 -0.000489  0.054751 -0.012180   
     46       -0.002215  0.027569  0.021476  0.003874 -0.001690  0.012346   
     47        0.000369  0.000962 -0.005902  0.001067  0.013055  0.015432   
     48       -0.012003 -0.001969  0.030976 -0.000712  0.009795 -0.006842   
     49        0.001416  0.003348 -0.003685 -0.002267 -0.006733  0.000575   

timeLag             248       249       250  
date stocksID                                
250  0        -0.010776 -0.013002 -0.018647  
     1         0.012173 -0.022280 -0.008254  
     2        -0.006044 -0.013629 -0.008404  
     3        -0.008568 -0.006981 -0.022734  
     4        -0.007991 -0.008315 -0.024546  
...                 ...       ...       ...  
753  45        0.019142 -0.023184 -0.001315  
     46        0.007502  0.011425 -0.001580  
     47        0.023533 -0.022811 -0.001053  
     48        0.000464  0.015894 -0.003321  
     49        0.000735 -0.020917 -0.004808  

[25200 rows x 250 columns]

# Function

In [11]:
def fitBeta(A):
    X_train_reshape2 = torch.tensor(X_train_reshape.values)
    predictors = torch.mm(X_train_reshape2 , A) # the dataframe of the 10 factors created from A with the (date, stock) in index

    targets = Y_train.T.stack()
    targets = torch.tensor(targets.values)
    targets = targets[:,None]

    beta = torch.linalg.inv(torch.mm(predictors.T, predictors))
    beta = torch.mm(beta, predictors.T)
    beta = torch.mm(beta, targets)
    

    
    return beta

def metric(df_y_true, df_y_pred):
    """ Compute metric. """
    if df_y_pred is None:  # If the y_pred has only zeroes, the metric is set to -1.
        return -1.0
    
    y_true = df_y_true.T
    y_pred = df_y_pred.T
    
    y_true = y_true.div(y_true.pow(2.0).sum(1).pow(0.5), 0)
    y_pred = y_pred.div(y_pred.pow(2.0).sum(1).pow(0.5), 0)

    mean_overlap = (y_true * y_pred).sum(1).mean()

    return mean_overlap   

def metric_train(X, beta): 
    
    X_train_reshape2 = torch.tensor(X_train_reshape.values)
    beta = beta.reshape(10,1)
    Ypred = torch.mm(X_train_reshape2, X)
    Ypred = torch.mm(Ypred, beta)
    Ypred = Ypred.reshape(504,50).T 
    Ytrue = torch.tensor(Y_train.values)
    
    for k in range(504):
        Ytrue[:,k] = Ytrue[:,k].clone()/torch.linalg.norm(Ytrue[:,k].clone())

        Ypred[:,k] = Ypred[:,k].clone()/torch.linalg.norm(Ypred[:,k].clone())

    
    meanOverlap = torch.mean(torch.sum(Ytrue*Ypred,dim =  0))

    return 0.2-meanOverlap





# Function Submission

In [12]:
def transform_submission_to_ypred(df_A,df_beta, x_test, y_test):
    """ Transform submission output (A, beta) into predicted returns S_t."""
    df_A = df_A.to_numpy()
    df_beta = df_beta.to_numpy()
    
    A = df_A.reshape((250, 10))
    beta = df_beta.reshape(10)

    E = pd.DataFrame(A.T @ A - np.eye(10)).abs()  

    # check orthogonality of A
    if any(E.unstack() > 1e-6): 
        return None

    x_test = x_test.T
    y_test = y_test.T

    x_test = x_test[y_test.columns]

    x_test_reshape = pd.concat([x_test.shift(i+1).stack(dropna=False) for i in range(250)], 1).dropna()
    y_pred = (x_test_reshape @ A @ beta).unstack()

    return y_pred.T

def parametersTransform(A, beta, D=250, F=10):
    
    if A.shape != (D, F):
        print('A has not the good shape')
        return
    
    if beta.shape[0] != F:
        print('beta has not the good shape')
        return        
    
    output = np.hstack( (np.hstack([A.T, beta.reshape((F, 1))])).T )
    
    return output

def fitBeta_df(A):
    predictors = X_train_reshape @ A # the dataframe of the 10 factors created from A with the (date, stock) in index
    targets = Y_train.T.stack()
    beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets
    
    return beta.to_numpy()

# Training

In [13]:
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## First model

In [ ]:
class descente():
    def __init__(self):
        super(descente, self).__init__()
        #self.A = torch.tensor(A_QRT, requires_grad = True)
        self.A = torch.rand((250,10), requires_grad = True)

    
    def parameters(self):
      return([self.A])

    def forward(self):
        return torch.linalg.qr(self.A, mode='reduced')[0]

In [ ]:
model = descente()

A = torch.zeros((250,10), requires_grad = True)
beta =  fitBeta(A)

n_epochs = 1000
lr = 0.5

optimizer = torch.optim.Adam(model.parameters(), lr)


In [ ]:
total_loss = 0.0
min_loss = 0.2
Mat_fin = torch.rand(250,10)
Loss_mat = []
Loss_test = []
for j in range(n_epochs):
  
  Mat = model.forward().double()
  Mat.to(device)

  beta  = Mat.clone().detach()
  beta.to(device)
  beta =  fitBeta(beta)
  

  #loss = metric_train(Mat, beta  )
  loss =  metric_train(Mat, beta)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if loss.item() < min_loss:
    min_loss = loss.item()
    Mat_fin = Mat.clone().detach().numpy()


 
  
  total_loss += loss.item()
  if j >0:
    Loss_test.append(metric_train(Mat.detach(), beta))
    Loss_mat.append(loss.item())

  if j%100 ==0:
    abscisse = np.linspace(1,len(Loss_mat), len(Loss_mat))
    plt.plot(abscisse, Loss_mat)
    plt.plot(abscisse, Loss_test)

    plt.show()
    #Loss_mat = []
    print('At epochs '+str(j+1)+' : '+str(total_loss/(j+1)))
    print('min_loss : '+str(min_loss))
    Mat_fin = pd.DataFrame(Mat_fin)
    Mat_fin.to_csv('./Challenge/random'+str(j))

# Second model

In [ ]:
class descente():
    def __init__(self):
        super(descente, self).__init__()
        #self.A = torch.tensor(A_QRT, requires_grad = True)
        self.A = torch.rand((250,10), requires_grad = True)
        self.B = torch.rand((10), requires_grad = True)

    def parameters(self):
      return([self.A, self.B])

    def forward(self):
        return torch.linalg.qr(self.A, mode='reduced')[0], self.B

In [ ]:
model = descente()
n_epochs = 1000

optimizer = torch.optim.Adam(model.parameters(), lr= 0.005)
total_loss = 0.0
min_loss = 0.2
Mat_fin = torch.rand(250,10)
beta_fin = torch.rand(10)
Loss_mat = []
for j in range(n_epochs):
  
  Mat, beta = model.forward()
  Mat.to(device)
  Mat = Mat.double()
  #beta  = Mat.clone().detach()
  beta.to(device)
  beta = beta.double()
  


  loss =  metric_train(Mat, beta)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if loss.item() < min_loss:
    min_loss = loss.item()
    Mat_fin = Mat.clone().detach().numpy()
    beta_fin = beta.clone().detach().numpy()


 
    
  total_loss += loss.item()
  if j >0:

    Loss_mat.append(loss.item())

  if j%50 ==0:
    abscisse = np.linspace(1,len(Loss_mat), len(Loss_mat))
    plt.plot(abscisse, Loss_mat)
    plt.show()
    #Loss_mat = []
    print('At epochs '+str(j+1)+' : '+str(total_loss/(j+1)))
    print('min_loss : '+str(min_loss))
    Mat_fin = pd.DataFrame(Mat_fin)
    Mat_fin.to_csv('./Challenge/A'+str(j))

    beta_fin = pd.DataFrame(beta_fin)
    beta_fin.to_csv('./Challenge/beta'+str(j))
    

# Submission 

In [ ]:
pathA = ''
pathOutput = ''

A = pd.read_csv('./Challenge/'+pathA, index_col=0, sep=',').to_numpy()
#beta = pd.read_csv(PATH_GC+'/Challenge/beta1050', index_col=0, sep=',').to_numpy()

beta = fitBeta_df(A)
output = parametersTransform(A, beta)
pd.DataFrame(output).to_csv('./Challenge/'+pathOutput+'.csv')